## 1. Initializing PyMISP and connecting to the MISP instance.

In [1]:
# The URL of the MISP instance to connect to

misp_url = 'https://misp-instance.com.au/'
# Can be found in the MISP web interface under ||
# http://+MISP_URL+/users/view/me -> Authkey
misp_key = 'api-key'
# Should PyMISP verify the MISP certificate
misp_verifycert = False

In [2]:
#Initialize PyMISP

from pymisp import PyMISP
misp = PyMISP(misp_url, misp_key, debug=False)

The version of PyMISP recommended by the MISP instance (2.4.188) is newer than the one you're using now (2.4.167). Please upgrade PyMISP.


## 2. Pulling events using specific tag and date

Tags can be 'VirusTotal', 'URLscan', 'AUSCERT', 'Community sourced', 'Third party sourced', 'CTIS - High Confidence', 'AlienVault OTX', 'tlp:green, tlp:amber, tlp:white'

These are also searchable fields:
id: 33980
orgc_id: 2
org_id: 2
date: 2024-03-05
threat_level_id: 2
info: 2024-03-05 - VirusTotal Ransomware daily IoC's
published: False
uuid: c5ce2bc0-3356-4247-956f-8791b662dd42
attribute_count: 1723
analysis: 2
timestamp: 1709593257
distribution: 2
proposal_email_lock: False
locked: False
publish_timestamp: 0
sharing_group_id: 0
disable_correlation: False
extends_uuid: 
protected: None
event_creator_email: test@auscert.org.au

In [4]:
#Search events that tagged with 'VirusTotal' and from 2024-04-09 to 2024-04-10

events = misp.search(date=['2024-04-09', '2024-04-10'], tags=['tlp:green'], metadata=False, pythonify=True)
print(events)
print("After adding Virustotal tag: ")
#From all tlp:green tagged events, I'm now interested in Virustotal tagged event.
filtered_events = [event for event in events if any(tag.name == 'VirusTotal' for tag in event.tags)]
print(filtered_events)

#we can keep adding more filters until we get what we want.

[<MISPEvent(info=2024-04-09 - Potentially malicious URLs submitted from Australian IP in URLscan.io), <MISPEvent(info=2024-04-09 - VirusTotal Ransomware daily IoC's), <MISPEvent(info=2024-04-10 - Potentially malicious URLs submitted from Australian IP in URLscan.io), <MISPEvent(info=2024-04-10 - VirusTotal Ransomware daily IoC's)]
After adding Virustotal tag: 
[<MISPEvent(info=2024-04-09 - VirusTotal Ransomware daily IoC's), <MISPEvent(info=2024-04-10 - VirusTotal Ransomware daily IoC's)]


In [22]:
#Search events that tagged with 'AUSCERT' and from 48hrs ago to 24hrs ago.

events = misp.search(timestamp=['48h', '24h'], tags=['AUSCERT'], metadata=False, pythonify=True)

In [39]:
#Search events that tagged with 'URLscan' and since 2024-02-19

events = misp.search(date='2024-04-19', tags=['URLScan'], metadata=False, pythonify=True)

In [30]:
#Search events that tagged with 'tlp:green' and in last 48hrs

events = misp.search(timestamp='48h', tags=['tlp:green'], metadata=False, pythonify=True)

In [33]:
#Search events using system time

from datetime import datetime, date, timedelta
from dateutil.parser import parse

today = int(datetime.today().timestamp())
yesterday = int((datetime.today() - timedelta(days=1)).timestamp())

events = misp.search(timestamp=yesterday,tags=['tlp:amber'], metadata=False, pythonify=True)

In [43]:
#Search events with eventid

events = misp.search(eventid=[33980], metadata=False, pythonify=True)

In [5]:
#Printing search results
print(events)

[<MISPEvent(info=2024-04-09 - Potentially malicious URLs submitted from Australian IP in URLscan.io), <MISPEvent(info=2024-04-09 - VirusTotal Ransomware daily IoC's), <MISPEvent(info=2024-04-10 - Potentially malicious URLs submitted from Australian IP in URLscan.io), <MISPEvent(info=2024-04-10 - VirusTotal Ransomware daily IoC's)]


## 3. Printing MISP attributes from events that found by our previous search.

In [10]:
# If search found multiple events, this for loop will check each one of them.
for event in filtered_events:
    
    #printing informations about the event
    print(event)
    print("Event ID: " + str(event.id))
    print("Event Date: " + str(event.date))
    print("Attribute count: " + str(event.attribute_count))
    
    #MISP event can have attributes and also Objects. This part of code will cover both of them.
    Object_list = event.Object    
    Attribute_list = event.Attribute
    final_list = []
    #if event has objects
    if Object_list:
        for misp_object in Object_list:
            if hasattr(misp_object, "Attribute"):
                #print(misp_object.Attribute)
                attr_list = misp_object.Attribute
                for attr in attr_list:
                    #if attr.object_relation=="md5" and attr.to_ids == True:
                        #final_list.append(attr.value)
                        #print(attr.value)
                    if attr.to_ids == True:
                        final_list.append(attr.value)
                        #print(attr.value)
                #print(attr)
            else:
                print(f"One of the objects does not have 'Object' or 'Attributes' attribute.")
    else:
        print("No object in this event.")
    
    #if event has attributes
    if Attribute_list:
        for attribute in Attribute_list:
            #print(attribute.type)
            if attribute.to_ids == True:
                final_list.append(attr.value)
                #print(attribute.value)
    else:
        print("No attribute in this event")
    
    #Writing into a text file
    import json
    output_data = {'Attributes': final_list}
    json_output = json.dumps(output_data)
    #print(json_output)
    with open(str(event.id)+'.txt', 'w') as file:
        file.write(json_output)

<MISPEvent(info=2024-04-09 - VirusTotal Ransomware daily IoC's)
Event ID: 51354
Event Date: 2024-04-09
Attribute count: 2107
<MISPEvent(info=2024-04-10 - VirusTotal Ransomware daily IoC's)
Event ID: 51356
Event Date: 2024-04-10
Attribute count: 1963


### You can also choose which type of attribute you want to filter

#### For all atrributes with IDS tag:

if attr.to_ids == True:

#### For md5, sha1, sha256 hash or URL with IDS tag:

if attr.object_relation=="md5" and attr.to_ids == True:

if attr.object_relation=="sha1" and attr.to_ids == True:

if attr.object_relation=="sha256" and attr.to_ids == True:

if attr.object_relation=="URL" and attr.to_ids == True:

In [9]:
#to know recently deleted attributes, we can use this code. 
# Searching any edited or deleted attributes from MISP log.
from datetime import datetime, timedelta

date_from = (datetime.now() - timedelta(days=7)).strftime('%Y-%m-%d')
print(date_from)
date_to = datetime.now().strftime('%Y-%m-%d')

logs = misp.search_logs(model='Attribute', limit=5, action='delete', created=date_from)

for log in logs:
    print(log['Log'])
    action = log['Log']['action']
    event_id = log['Log']['model_id']
    user_email = log['Log']['email']
    timestamp = log['Log']['created']
    if action == 'edit':
        print(f"Event ID: {event_id} was edited by {user_email} at {timestamp}")
    if action == 'delete':
        print(f"Event ID: {event_id} was deleted by {user_email} at {timestamp}")
#print(logs)

2024-05-27
{'id': '27674591', 'title': 'Attribute (6142369) from Event (51433): Other/text Severity MEDIUM because it was considered trojan.', 'created': '2024-05-29 13:46:56', 'model': 'Attribute', 'model_id': '6142369', 'action': 'delete', 'user_id': '5', 'change': '', 'email': 'duke@auscert.org.au', 'org': 'AusCERT', 'description': 'Attribute "Severity MEDIUM because it was considered trojan." (6142369) deleted by User "duke@auscert.org.au" (5).', 'ip': '10.209.209.169'}
Event ID: 6142369 was deleted by duke@auscert.org.au at 2024-05-29 13:46:56
{'id': '27674630', 'title': 'Attribute (6142315) from Event (51433): Other/text Severity MEDIUM because it was considered trojan.', 'created': '2024-05-29 15:29:09', 'model': 'Attribute', 'model_id': '6142315', 'action': 'delete', 'user_id': '5', 'change': '', 'email': 'duke@auscert.org.au', 'org': 'AusCERT', 'description': 'Attribute "Severity MEDIUM because it was considered trojan." (6142315) updated by User "duke@auscert.org.au" (5).', '

## 4. Searching Indicator from MISP

In [47]:
#this one is useful for finding IoC's from the MISP
complex_query = misp.build_complex_query(or_parameters=['af696f4bd3d739398070aa247888d55ccd30b272'])
events = misp.search(value=complex_query)

for e in events:
    print(e['Event']['id'])
    print(e['Event']['info'])

33980
2024-03-05 - VirusTotal Ransomware daily IoC's
